In [1]:
import json
import pandas as pd
import re
import numpy as np

In [ ]:
import numpy as np

### json 파일 불러오기

In [5]:
data = []
with open('..\\데이터\\Epinion\\epinions_2.json', 'r', encoding='UTF-8') as f:
    for line in f:
        line = json.dumps(line)
        data.append(json.loads(line))
item = pd.DataFrame(data)

In [6]:
temp = []
for index,value in item.iterrows():
    temp.append(json.loads(value[0].replace("'","\"")))

In [7]:
df = pd.DataFrame(temp)

## pickle로 저장

In [ ]:
import pickle 
with open('.\\데이터\\Epinion\\epinions_2.pickle', 'wb') as f:
    pickle.dump(df, f)   

In [107]:
df.head()

,user,stars,time,item,review
0,fgb59h,4.0,1027296000,Minolta_QMS_PagePro_1250E_Printers,i got this printer from minolta as a warranty ...
1,bucho_ky,2.0,1201305600,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,although i love my vaio especially the gorgeou...
2,redp944,4.0,1118016000,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,this is my first laptop and i bought it about ...
3,spongebag7,4.0,1149292800,pr-Durabrand_CD-85_Personal_CD_Player,just recently my sony dej001 walkman died on m...
4,pyros7,5.0,1012262400,pr-Sony_KV_36FS12__Standard_Televisions,well when i graduated from college it didnt ta...


In [12]:
print("참여하는 유저의 수 : ", len(df['user'].unique()))
print("참여하는 item의 수 : ", len(df['item'].unique()))
print("interaction의 수 : ", len(df))

참여하는 유저의 수 :  116260
참여하는 item의 수 :  41269
interaction의 수 :  188478


In [8]:
import pickle
with open('.\\데이터\\Epinion\\epinions_2.pickle', 'rb') as f:
    df = pickle.load(f)

In [11]:
trust = pd.read_table('..\\데이터\\Epinion\\network_trust.txt', sep = ' ', header=None,names=['give', 'trust', 'take'])

In [64]:
trust.head()

,give,trust,take
0,lavenderfruit,trust,kristinafh
1,shippo225,trust,sallyg8
2,aryeh7000,trust,liteair
3,chiquita,trust,magenta321
4,jeffsullivan,trust,rpllingrock


In [12]:
trust_user = set()
trust_user.update(trust['give'].unique())
trust_user.update(trust['take'].unique())

In [13]:
review_user = set(df['user'].unique())

In [15]:
print('review를 남기지 않은 user의 수 : ' ,len(trust_user-review_user))
print('link가 없는 user의 수 : ', len(review_user-trust_user))

review를 남기지 않은 user의 수 :  66916
link가 없는 user의 수 :  98645


In [17]:
user_set = set()
user_set.update(trust['give'].unique())
user_set.update(trust['take'].unique())
user_set.update(df['user'].unique())

In [92]:
print("Edge에 참여하는 user의 수 : ", len(user_set))

Edge에 참여하는 user의 수 :  183176


In [99]:
print("interaction에 참여하는 user의 수 : ", len(df['user'].unique())) 
print("Network에 참여하는 user의 수 : ", 84531)
print("Epinion에서의 Unique user의 수(interaction+network) : ", 183176)

interaction에 참여하는 user의 수 :  116260
Network에 참여하는 user의 수 :  84531
Epinion에서의 Unique user의 수(interaction+network) :  183176


### item에 index부여하고 저장

In [18]:
map_list = []
item_list = df['item'].unique()
for i, item in enumerate(item_list):
            map_list.append([item, i])
map_df = pd.DataFrame(map_list, columns=['org_id', 'remap_id'])

In [41]:
map_df.to_csv('./git/item_list.txt', sep = '\t', index = False)

## user에 index를 부여하고 저장

In [19]:
map_user = []
for i, user in enumerate(user_set):
    map_user.append([user, i])
del map_user[0]
map_user_df = pd.DataFrame(map_user, columns = ['org_id', 'remap_id'])

In [104]:
map_user_df.to_csv('./git/user_list.txt', sep = '\t', index =False)

In [20]:
map_user_df.head()

,org_id,remap_id
0,Spoon,1
1,petenme,2
2,sniklu,3
3,joedemeo,4
4,shoppersdiary,5


In [21]:
map_user_dict = dict(zip(map_user_df.org_id,map_user_df.remap_id))

In [22]:
map_item_dict = dict(zip(map_df.org_id,map_df.remap_id))

In [38]:
u_i_rating = df[['user', 'item']]

In [39]:
u_i_rating.head()

,user,item
0,fgb59h,Minolta_QMS_PagePro_1250E_Printers
1,bucho_ky,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...
2,redp944,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...
3,spongebag7,pr-Durabrand_CD-85_Personal_CD_Player
4,pyros7,pr-Sony_KV_36FS12__Standard_Televisions


In [40]:
u_i_rating['user'] = u_i_rating['user'].map(lambda x: map_user_dict[x])
u_i_rating['item'] = u_i_rating['item'].map(lambda x: map_item_dict[x])

ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
u_i_rating = np.asarray(u_i_rating)

In [56]:
interaction = []
for user_id in sorted(map_user_dict.values()):
    match = u_i_rating[u_i_rating[:,0] == user_id]
    if len(match)>10:
        interaction.append([user_id]+list(match[:,1]))

In [88]:
print("10개 이상 review 남긴 User의 수 : ",len(interaction))

10개 이상 review 남긴 User의 수 :  1062


In [195]:
with open('./git/total_data.txt', 'w') as output:
    for line in interaction:
        output.write(" ".join(map(str,line)))
        output.write('\n')